<a href="https://colab.research.google.com/github/jaydeepthik/PyTorch-implementations/blob/master/cifar_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)


test_set = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False, num_workers=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0it [00:00, ?it/s]

100%|█████████▉| 170369024/170498071 [00:29<00:00, 15243986.53it/s]

Files already downloaded and verified


In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 3)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self ,X):
    X = self.pool(F.relu(self.conv1(X)))
    X = self.pool(F.relu(self.conv2(X)))
    X = X.view(-1, 16*5*5)
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return X
  
net = Net()


In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)

In [5]:
for epoch in range(3):
  running_loss = 0.0
  
  for i, data in enumerate(train_loader,0):
    inputs, labels = data
    
    #zero optimizer gradients
    optimizer.zero_grad()
    
    #forward->backward->optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    running_loss+= loss.item()
    
    if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
          
print("finished training")

[1,  2000] loss: 2.257
[1,  4000] loss: 1.982
[1,  6000] loss: 1.798
[1,  8000] loss: 1.636
[1, 10000] loss: 1.580
[1, 12000] loss: 1.512
[2,  2000] loss: 1.447
[2,  4000] loss: 1.395
[2,  6000] loss: 1.379
[2,  8000] loss: 1.361
[2, 10000] loss: 1.304
[2, 12000] loss: 1.284
[3,  2000] loss: 1.227
[3,  4000] loss: 1.217
[3,  6000] loss: 1.198
[3,  8000] loss: 1.206
[3, 10000] loss: 1.189
[3, 12000] loss: 1.199
finished training


In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 58 %
